<a href="https://colab.research.google.com/github/JaNiAr2/JaNiAr2/blob/main/Python%20Final%3A%20US%20Electricity%20Generation%20(2001-2019).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Python Final Project: Energy Mix in the United States (2000 - 2019)**



Since the dawn of the industrial revolution in the mid-18th century, the combustion of fossil fuels has contributed to the rapid accumulation of greenhouse gases (GHGs) in our atmosphere,
thereby causing the gradual warming of our planet. Not all countries have contributed the same amount of GHGs. According to analysis from Our World in Data, since 1750 the United States,
China, and Russia have emitted the most GHG emissions, at 24%, 15%, and 7% respectively (Ritchie, 2025). With its disproportional impact on climate change, some argue that the United States has an equally disproportionate
responsibility to lead the fight against climate change by leading the way in their own domestic decarbonization. According to the Environmental Protection Agency, 24% of U.S. emissions came from
the electric power sector in 2022 (EPA), though this trend has been on a decline since 2007.

This project leverages U.S. energy generation data from 2001 to 2019 from all 50 states to understand and depict how the U.S. energy system has made progress in decarbonizing its
electricity production while diversifying into low carbon generation technologies. Additional, this analysis leverages information on state governorships to understand under which party's leadership changes
in the electrical system occured. This project dives into three states with a particualrly interesting history in their electricity systems. First, California, one of the most widely renowned leaders of
renewable energy generation. Second, West Virginia, with its long history of coal mining. Third, Texas, which boasts the Permian Basin as a massive source of natural gas, but has simultaneously been a strong
proponent of wind energy.

This project has the following structure:

**1. Data Setup and Preliminary Analysis**

**2. Analysis of the United States (2001-2019)**

**3. Analysis of California, West Virginia, and Texas (2001-2019)**

**4. Conclusion**

**5. Sources**

-----------------------------------------------------

**1. Data Setup and Preliminary Analysis**

In [ ]:
import kagglehub
import pandas as pd

# Download dataset
data = kagglehub.dataset_download("kevinmorgado/us-energy-generation-2001-2022")

import os

# See what files are inside the folder
os.listdir(data)

# Load the generation dataset
gen = pd.read_csv(os.path.join(data, "organised_Gen.csv"))

#delete & rename columns
gen = gen.drop(columns=["Unnamed: 0"])
gen = gen.drop(columns=["MONTH"])
gen = gen.rename(columns={"GENERATION (Megawatthours)": "GENERATION (MWh)"})
gen = gen[gen["YEAR"] != 2022]


Using Colab cache for faster access to the 'us-energy-generation-2001-2022' dataset.


The data from the Energy Information Agency provides state-level data on electric generators from 2001 to 2021. Figure 1 below shows the breakdown of generation between the five types of generators provided in the data that exist throughout the United States. The vast majority of generated electricity comes from electric utilites (60%) and independent power producers (32%), while only marginal amounts are supplied by combined heat and power generators.

In [ ]:
import pandas as pd
from IPython.display import display, Markdown

# --- Detect the generation column dynamically ---
gen_col = None
for c in gen.columns:
    if "GENERATION" in c.upper():
        gen_col = c
        break
if gen_col is None:
    raise KeyError("Could not find a generation column (expected 'GENERATION (MWh)' or similar).")

# --- Remove "Total Electric Power Industry" rows ---
gen = gen[~gen["TYPE OF PRODUCER"].astype(str).str.strip().eq("Total Electric Power Industry")].copy()

# --- Aggregate total generation by producer type ---
producer_summary = (
    gen.groupby("TYPE OF PRODUCER", as_index=False)[gen_col]
      .sum()
      .rename(columns={
          "TYPE OF PRODUCER": "Type of Producer",
          gen_col: "Total Generation (MWh)"
      })
)

# --- Convert to GWh ---
producer_summary["Total Generation (GWh)"] = producer_summary["Total Generation (MWh)"] / 1_000

# --- Compute share of total ---
total_gen = producer_summary["Total Generation (GWh)"].sum()
producer_summary["Share (%)"] = (
    100 * producer_summary["Total Generation (GWh)"] / total_gen
).round(2)

# --- Clean up ---
producer_summary = (
    producer_summary
    .drop(columns="Total Generation (MWh)")
    .sort_values("Total Generation (GWh)", ascending=False)
    .reset_index(drop=True)
)

# --- Display nicely ---
display(Markdown("### Figure 1. Types of Producers — Share of Total Electricity Generation (2001–2019, Excluding Total Industry)"))
display(producer_summary)


### Figure 1. Types of Producers — Share of Total Electricity Generation (2001–2019, Excluding Total Industry)

,Type of Producer,Total Generation (GWh),Share (%)
0,"Electric Generators, Electric Utilities",2.015204e+08,59.38
1,"Electric Generators, Independent Power Producers",1.112327e+08,32.78
2,"Combined Heat and Power, Electric Power",1.352813e+07,3.99
3,"Combined Heat and Power, Industrial Power",1.222189e+07,3.60
4,"Combined Heat and Power, Commercial Power",8.731632e+05,0.26


With the understanding of what types of entities are producing energy in this dataset, Figure 1.2 highlights the energy sources on which the electricity generators in Figure 1 run, separated in two years: 2001 and 2019. The majority of generators run on fossil fuels, both in 2001 and 2019, with natural gas, petroleum, and coal leading the way.

In [ ]:
import pandas as pd
from IPython.display import display, Markdown

# Ensure YEAR numeric
gen["YEAR"] = pd.to_numeric(gen["YEAR"], errors="coerce")

# Keep only 2001 and 2019
subset = gen[gen["YEAR"].isin([2001, 2019])].copy()

# Exclude the 'Total' category before counting
subset_no_total = subset[~subset["ENERGY SOURCE"].astype(str).str.strip().str.lower().eq("total")]

# Build counts table (records per energy source per year)
tbl = (
    pd.crosstab(subset_no_total["ENERGY SOURCE"], subset_no_total["YEAR"])
      .reindex(columns=[2001, 2019], fill_value=0)
      .rename(columns={2001: "2001", 2019: "2019"})
      .astype(int)
)

# Compute a proper Total row as the sum across sources
total_row = pd.DataFrame({"ENERGY SOURCE": ["Total"], "2001": [tbl["2001"].sum()], "2019": [tbl["2019"].sum()]})
total_row = total_row.set_index("ENERGY SOURCE")

# Sort by 2001 desc and append the Total row last
tbl = tbl.sort_values(by="2001", ascending=False)
tbl = pd.concat([tbl, total_row])

# Reset index for display
tbl = tbl.reset_index().rename(columns={"ENERGY SOURCE": "Energy Source"})

# Display
display(Markdown("### Figure 1.2 Energy Sources — Count of Generators by Energy Source (2001 vs 2019)"))
display(tbl)


### Figure 1.2 Energy Sources — Count of Generators by Energy Source (2001 vs 2019)

,Energy Source,2001,2019
0,Natural Gas,2391,2539
1,Petroleum,2177,2376
2,Coal,1622,1588
3,Other Biomass,1310,1812
4,Hydroelectric Conventional,1206,1224
5,Other,887,1390
6,Wood and Wood Derived Fuels,804,1111
7,Nuclear,465,389
8,Other Gases,454,420
9,Wind,267,1080


The energy generation mix has changed dramatically in the United States as the cost of renewables has dropped, and through the shale revolution in the mid 2000s. Figure 1.3 below shows total generation amounts based on generation type in both 2001 and 2019, as well as the average amount of annual generation per generator.

These initial finding confirm the shift that occured in the United States away from coal generation, which was the dominating fossil fuel in 2001, to natural gas, which two decades later dominates U.S. electricity generation.

It's also notable to see the dramatic increase in both wind and solar PV generation. A key component of solar PV generation that is missing, however, is distributed generation. As noted in Figure 1, this data set only includes large-scale generation from electric utilities and intependent power producers. Much of the country's PV generation comes from rooftop solar and other distributed solar, particularly in states like California where new homes are mandated to build solar PV on their rooftops.

It is worth explaining why pumped storage has negative generation values. Pumped storage is a technology during which water is pumped uphill into a reservoir during off-peak hours when electricity is cheap in order for the water to be released downhill to generate electricity during peak hours where demand and prices are high. Generally it takes more electricity to pump the water into its uphill reservoir than it generates upon its release, therefore leading to overall net-negative generation values.

Overall, Figure 1.3 shows that electric generation increased by about 10% between 2001 and 2019, and that the overall mix of electricity generation changed dramatically, with coal losing its status as market leader. It is also worth noting the differences in average generation, with coal and nuclear generators, on average, generating much larger quantities of electricity per generator than other forms of generators.

In [ ]:
import pandas as pd
from IPython.display import display, Markdown

# --- Detect the generation column (handles different names) ---
gen_col = None
for c in gen.columns:
    if "GENERATION" in c.upper():
        gen_col = c
        break
if gen_col is None:
    raise KeyError("Could not find a generation column (expected 'GENERATION (MWh)' or similar).")

# Ensure YEAR numeric
gen["YEAR"] = pd.to_numeric(gen["YEAR"], errors="coerce")

# Keep only 2001 and 2019; drop 'Total' source rows before summing
subset = gen[gen["YEAR"].isin([2001, 2019])].copy()
subset = subset[~subset["ENERGY SOURCE"].astype(str).str.strip().str.lower().eq("total")]

# --- 1️⃣ Total generation (GWh) in 2001 and 2019 ---
by_src = (
    subset.groupby(["YEAR", "ENERGY SOURCE"], as_index=False)[gen_col]
          .sum()
)
by_src["GWh"] = pd.to_numeric(by_src[gen_col], errors="coerce") / 1_000

tbl = (
    by_src.pivot(index="ENERGY SOURCE", columns="YEAR", values="GWh")
         .reindex(columns=[2001, 2019])
         .fillna(0.0)
         .rename(columns={2001: "2001 (GWh)", 2019: "2019 (GWh)"})
)

# --- 2️⃣ Average generation per generator (2001–2019) ---
# Use all years (2001–2019), exclude 'Total' row
all_years = gen[
    gen["YEAR"].between(2001, 2019)
    & ~gen["ENERGY SOURCE"].astype(str).str.strip().str.lower().eq("total")
].copy()
all_years["GWh"] = pd.to_numeric(all_years[gen_col], errors="coerce") / 1_000

# total generation / count (proxy for generators)
avg_gen = (
    all_years.groupby("ENERGY SOURCE")
             .agg({"GWh": ["sum", "count"]})
)
avg_gen.columns = ["Total_GWh_2001_2019", "Count"]
avg_gen["Average Generation per Generator (GWh, 2001–2019)"] = avg_gen["Total_GWh_2001_2019"] / avg_gen["Count"]
avg_gen = avg_gen[["Average Generation per Generator (GWh, 2001–2019)"]]

# Merge with Figure 3 table
tbl = tbl.join(avg_gen, how="left")

# --- 3️⃣ Add Total row and sort by 2001 descending ---
total_row = pd.DataFrame({
    "ENERGY SOURCE": ["Total"],
    "2001 (GWh)": [tbl["2001 (GWh)"].sum()],
    "2019 (GWh)": [tbl["2019 (GWh)"].sum()],
    "Average Generation per Generator (GWh, 2001–2019)": [None]  # not meaningful for total
}).set_index("ENERGY SOURCE")

tbl = tbl.sort_values(by="2001 (GWh)", ascending=False)
tbl = pd.concat([tbl, total_row])

# --- 4️⃣ Format numbers ---
tbl["2001 (GWh)"] = tbl["2001 (GWh)"].round(0).astype(int).map(lambda x: f"{x:,}")
tbl["2019 (GWh)"] = tbl["2019 (GWh)"].round(0).astype(int).map(lambda x: f"{x:,}")
tbl["Average Generation per Generator (GWh, 2001–2019)"] = tbl["Average Generation per Generator (GWh, 2001–2019)"].apply(
    lambda x: f"{x:,.1f}" if pd.notnull(x) else ""
)

# --- 5️⃣ Display ---
tbl = tbl.reset_index().rename(columns={"ENERGY SOURCE": "Energy Source"})
display(Markdown("### Figure 1.3 Energy Sources — Total Generation (GWh) in 2001 vs 2019, and Average Generation per Generator (2001–2019)"))
display(tbl)


/tmp/ipython-input-647347365.py:63: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



### Figure 1.3 Energy Sources — Total Generation (GWh) in 2001 vs 2019, and Average Generation per Generator (2001–2019)

,Energy Source,2001 (GWh),2019 (GWh),"Average Generation per Generator (GWh, 2001–2019)"
0,Coal,"3,807,912","1,929,914","2,027.7"
1,Nuclear,"1,537,653","1,618,819","3,877.1"
2,Natural Gas,"1,278,258","3,171,434",819.8
3,Hydroelectric Conventional,"433,922","575,747",441.8
4,Petroleum,"249,760","36,876",48.2
5,Wood and Wood Derived Fuels,"70,400","77,087",80.2
6,Other Biomass,"29,096","37,927",23.6
7,Geothermal,"27,481","30,945",268.0
8,Other,"23,813","26,669",23.3
9,Other Gases,"18,079","25,182",47.0


To simplify the analysis moving forward, the following energy sources will be group into a category category called "Other Sources."
(1) Wood and Wood Derived Fuels
(2) Other Biomass
(3) Other
(4) Other Gases
(5) Pumped Storage.


In [ ]:
# --- Group selected energy sources into "Other Sources" ---
# Define the list of sources to merge
other_sources = [
    "Wood and Wood Derived Fuels",
    "Other Biomass",
    "Other",
    "Other Gases",
    "Pumped Storage"
]

# Create a new column 'ENERGY SOURCE Grouped'
gen["ENERGY SOURCE Grouped"] = gen["ENERGY SOURCE"].apply(
    lambda x: "Other Sources" if str(x).strip() in other_sources else x
)

# Optional: verify grouping
print("✅ Energy sources successfully grouped. Unique categories now:")
print(sorted(gen["ENERGY SOURCE Grouped"].unique()))


✅ Energy sources successfully grouped. Unique categories now:
['Coal', 'Geothermal', 'Hydroelectric Conventional', 'Natural Gas', 'Nuclear', 'Other Sources', 'Petroleum', 'Solar Thermal and Photovoltaic', 'Total', 'Wind']


In an attempt to understand how changes in state politics may affect the trajectory of fossil fuels and renewables, this analysis leverages data that shows the governors in the 50 U.S. states, as well as their party affilation. The CSV file can be found here: https://www.openicpsr.org/openicpsr/project/102000/version/V3/view

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

gov = pd.read_csv('/content/drive/MyDrive/Coding/Data/US-Governors.csv')
gov.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,governor,state,time_in_office,party,year
0,Jeff Colyer,Kansas,2018 - 2019,Republican,2018
1,Jeff Colyer,Kansas,2018 - 2019,Republican,2019
2,Eric Greitens,Missouri,2017 - 2018,Republican,2017
3,Eric Greitens,Missouri,2017 - 2018,Republican,2018
4,Chuck Morse,New Hampshire,2017 - 2017,Republican,2017


In [ ]:
# Capitalize only the first letter of each word in the column names
gov.columns = [col.strip().title() for col in gov.columns]

print("Updated column names:")
print(list(gov.columns))

#Readability

gov.columns = [col.strip().replace('_', ' ').title() for col in gov.columns]
print("Updated column names:", list(gov.columns))

#Is "Year" numeric?
gov["Year"].dtype

Updated column names:
['Governor', 'State', 'Time_In_Office', 'Party', 'Year']
Updated column names: ['Governor', 'State', 'Time In Office', 'Party', 'Year']


dtype('int64')

In [ ]:
# Define the list of 50 official U.S. states
us_50_states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
    'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
    'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
    'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
    'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina',
    'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
    'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
    'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'
]

before_rows = len(gov)

# Keep only rows where the State is one of the 50 states
gov = gov[gov["State"].isin(us_50_states)].copy()

after_rows = len(gov)

print(f"Removed {before_rows - after_rows} non-state rows. Remaining: {after_rows}")
print("Unique states now:", len(gov['State'].unique()))

# Show the cleaned state list
unique_states = sorted(gov["State"].unique())
print("\nCleaned state list:")
print(unique_states)

Removed 318 non-state rows. Remaining: 12027
Unique states now: 50

Cleaned state list:
['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


**2. Analysis of the United States (2001-2019)**

Figures 2.1 - 2.4 below provide visual overview of how the U.S. energy mix has changed in the 21st century. Coal was the dominant source of electricity in 2021, boasting 50% market capture in 2001, which declined to 23% by 2019. Nuclear's aggregate generation doesn't change much, which highlights the fact that nuclear energy has struggled to attract new investment in the 21st century due to a history of construction delays and cost overruns which makes them a risky investment with massive uprfront costs.

Figure 2.2 highlights year-over-year changes, and several years following 2010 where annual growth in solar PV generation was in the double digits. On the other hand, fossil fuel generation based on coal or petroleum saw predominantly years of decline, or negative growth, where their annual generation amounts dropped year-over-year.

Since it is difficult to spot renewables and low-carbon generation in Figures 2.1 and 2.3, Figure 2.4 specifically looks at the role of those technologies in terms of aggregate annual generation. It highlights the fact that U.S. wind generation took dramatically starting in 2005, with solar taking off in 2013. Hydro and geothermal, however, remained mostly stagnant in the analyzed time period.

Importantly, upon verifying this data with the Interational Energy Agency's data, there is strong overlap on trends and patterns among these energy sources in the 2001-2019 time frame (IEA).

In [ ]:
import pandas as pd
import plotly.express as px

# --- 0) Detect the generation column (handles different names) ---
gen_col = None
for c in gen.columns:
    if "GENERATION" in c.upper():
        gen_col = c
        break
if gen_col is None:
    raise KeyError("Could not find a generation column (expected 'GENERATION (MWh)' or similar).")

# --- 1) Ensure grouped column exists (Other Sources) ---
if "ENERGY SOURCE Grouped" not in gen.columns:
    other_sources = {
        "wood and wood derived fuels",
        "other biomass",
        "other",
        "other gases",
        "pumped storage"
    }
    gen["ENERGY SOURCE Grouped"] = gen["ENERGY SOURCE"].apply(
        lambda x: "Other Sources" if str(x).strip().lower() in other_sources else x
    )

# --- 2) Filter: years & exclude 'Total' energy source rows ---
us = gen[
    gen["YEAR"].between(2001, 2019)
    & ~gen["ENERGY SOURCE"].astype(str).str.strip().str.lower().eq("total")
].copy()

# --- 3) Convert to GWh ---
us["GENERATION_GWH"] = pd.to_numeric(us[gen_col], errors="coerce") / 1_000

# --- 4) U.S. totals by YEAR x FUEL (grouped) ---
us_yearly = (
    us.groupby(["YEAR", "ENERGY SOURCE Grouped"], as_index=False)["GENERATION_GWH"]
      .sum()
      .sort_values(["YEAR", "ENERGY SOURCE Grouped"])
)

# --- 5) Overall U.S. totals (2001–2019) by FUEL (grouped) ---
us_total = (
    us.groupby("ENERGY SOURCE Grouped", as_index=False)["GENERATION_GWH"]
      .sum()
      .rename(columns={"GENERATION_GWH": "2001–2019 Total (GWh)"})
      .sort_values("2001–2019 Total (GWh)", ascending=False)
)

# Optional: pretty formatting
us_total_fmt = us_total.copy()
us_total_fmt["2001–2019 Total (GWh)"] = (
    us_total_fmt["2001–2019 Total (GWh)"].round(0).astype(int).map(lambda x: f"{x:,}")
)

# --- 6) Quick line chart: U.S. generation by fuel over time ---
fig = px.line(
    us_yearly,
    x="YEAR",
    y="GENERATION_GWH",
    color="ENERGY SOURCE Grouped",
    markers=True,
    labels={"YEAR": "Year", "GENERATION_GWH": "Generation (GWh)", "ENERGY SOURCE Grouped": "Energy Source"},
    title="Figure 2.1. United States — Electricity Generation by Source (GWh), 2001–2019"
)
fig.update_layout(template="plotly_white", xaxis=dict(dtick=1))
fig.show()

# ---------- (Re)build us_yearly if needed ----------
import pandas as pd
import plotly.express as px

if 'us_yearly' not in globals():
    # Detect generation column
    gen_col = next((c for c in gen.columns if "GENERATION" in c.upper()), None)
    if gen_col is None:
        raise KeyError("Generation column not found.")

    # Ensure grouped column exists (Other Sources)
    if "ENERGY SOURCE Grouped" not in gen.columns:
        other_sources = {
            "wood and wood derived fuels","other biomass","other","other gases","pumped storage"
        }
        gen["ENERGY SOURCE Grouped"] = gen["ENERGY SOURCE"].apply(
            lambda x: "Other Sources" if str(x).strip().lower() in other_sources else x
        )

    us = gen[
        gen["YEAR"].between(2001, 2019)
        & ~gen["ENERGY SOURCE"].astype(str).str.strip().str.lower().eq("total")
    ].copy()
    us["GENERATION_GWH"] = pd.to_numeric(us[gen_col], errors="coerce") / 1_000

    us_yearly = (
        us.groupby(["YEAR", "ENERGY SOURCE Grouped"], as_index=False)["GENERATION_GWH"]
          .sum()
          .sort_values(["YEAR", "ENERGY SOURCE Grouped"])
    )

# ---------- 2.2 Percentage change per source (YoY) ----------
us_yoy = us_yearly.sort_values(["ENERGY SOURCE Grouped", "YEAR"]).copy()
us_yoy["YoY %"] = (
    us_yoy.groupby("ENERGY SOURCE Grouped")["GENERATION_GWH"].pct_change() * 100
)

fig_22 = px.line(
    us_yoy.dropna(subset=["YoY %"]),
    x="YEAR", y="YoY %", color="ENERGY SOURCE Grouped", markers=True,
    labels={"YEAR":"Year","YoY %":"Year-over-Year Change (%)","ENERGY SOURCE Grouped":"Energy Source"},
    title="Figure 2.2 — U.S. Year-over-Year % Change in Generation by Source (2002–2019)"
)
fig_22.update_layout(template="plotly_white", xaxis=dict(dtick=1))
fig_22.show()

# ---------- 2.3 % of total (share of annual U.S. generation) ----------
share = us_yearly.copy()
share["US_TOTAL_GWH"] = share.groupby("YEAR")["GENERATION_GWH"].transform("sum")
share["Share (%)"] = 100 * share["GENERATION_GWH"] / share["US_TOTAL_GWH"]

fig_23 = px.area(
    share.sort_values(["YEAR","ENERGY SOURCE Grouped"]),
    x="YEAR", y="Share (%)", color="ENERGY SOURCE Grouped",
    labels={"YEAR":"Year","Share (%)":"Share of U.S. Generation (%)","ENERGY SOURCE Grouped":"Energy Source"},
    title="Figure 2.3 — U.S. Generation Mix (% of Total), 2001–2019"
)
fig_23.update_layout(template="plotly_white", xaxis=dict(dtick=1), yaxis=dict(range=[0,100]))
fig_23.show()

# ---------- 2.4 Low-carbon only (Geothermal, Hydro, Solar, Wind) ----------
LOW_CARBON = {"Geothermal", "Hydroelectric Conventional", "Solar Thermal and Photovoltaic", "Wind"}

lc = us_yearly[us_yearly["ENERGY SOURCE Grouped"].isin(LOW_CARBON)].copy()
fig_24 = px.line(
    lc.sort_values(["YEAR","ENERGY SOURCE Grouped"]),
    x="YEAR", y="GENERATION_GWH", color="ENERGY SOURCE Grouped", markers=True,
    labels={"YEAR":"Year","GENERATION_GWH":"Generation (GWh)","ENERGY SOURCE Grouped":"Energy Source"},
    title="Figure 2.4 — U.S. Low-Carbon Generation (Geothermal, Hydro, Solar, Wind), 2001–2019"
)
fig_24.update_layout(template="plotly_white", xaxis=dict(dtick=1))
fig_24.show()




**3. Analysis of California, West Virginia, and Texas (2001-2019)**

Figures 3.1 - 3.4 show the changes in the energy mix in California. The vertical bars overlayed on the graphs show changes in the state governorship. In 2003, for instance, Arnold Schwarzenegger (R) tstarted his tenure as governor. Interestingly, PV appears to have taken off after the end of Schwarzengger's term and the beginning of Democratic Jerry Brown's term. Unlike the national trend wherein wind grew more rapidly than solar PV, California's solar PV grew to become the state's leading low-carbon generation source. In total, however, it is still surprising to see how dependent the state is on fossil fuels, with natural gas representing 42% of the state's electricity generation. The omission of rooftop solar must also be mentioned here, as the state has a long history of providing incentives for homeowners and distributed generation developers to build small-scale solar.

In [ ]:
import pandas as pd
import plotly.express as px

# --------------------------- Helpers & setup ---------------------------
# 1) Detect generation column (handles 'GENERATION (MWh)' or similar)
def detect_gen_col(df):
    for c in df.columns:
        if "GENERATION" in c.upper():
            return c
    raise KeyError("Generation column not found (expected 'GENERATION (MWh)' or similar).")

# 2) Ensure grouped column exists: collapse to "Other Sources"
def ensure_grouped_column(df):
    if "ENERGY SOURCE Grouped" in df.columns:
        return df
    other_sources = {"wood and wood derived fuels","other biomass","other","other gases","pumped storage"}
    df = df.copy()
    df["ENERGY SOURCE Grouped"] = df["ENERGY SOURCE"].apply(
        lambda x: "Other Sources" if str(x).strip().lower() in other_sources else x
    )
    return df

# 3) California governors: keep start year of each governor (2001–2019)
def prep_governors_ca(gov_df):
    party_colors = {"Democratic": "blue", "Republican": "red", "Other": "gray"}
    g = gov_df[gov_df["State"].str.lower().eq("california")].copy()
    g = g[g["Year"].between(2001, 2019)]
    g = g.sort_values(["Governor", "Year"]).groupby("Governor", as_index=False).first()
    g["Color"] = g["Party"].map(party_colors).fillna("gray")
    return g

# 4) Add governor start-year lines + name labels
def add_governor_overlays(fig, gov_ca, yref="paper", y=0.97):
    for _, r in gov_ca.iterrows():
        year   = int(r["Year"])
        color  = r["Color"]
        name   = str(r["Governor"]).strip()
        party  = str(r.get("Party","Other")).strip()
        # line
        fig.add_vline(x=year, line_width=2, line_dash="dash", line_color=color)
        # label (placed in paper coords so it never clips)
        fig.add_annotation(
            x=year, y=y, yref=yref,
            text=f"{name} ({party[0]})",
            showarrow=False,
            font=dict(size=10, color="white"),
            bgcolor=color, bordercolor="black", borderwidth=0.5, borderpad=2,
            textangle=90, yanchor="top"
        )
    return fig

# --------------------------- Data prep (CA) ---------------------------
gen_col = detect_gen_col(gen)
gen = ensure_grouped_column(gen)

# Filter California, years, exclude 'Total' energy-source rows
ca = gen[
    gen["STATE"].str.upper().eq("CA")
    & gen["YEAR"].between(2001, 2019)
    & ~gen["ENERGY SOURCE"].astype(str).str.strip().str.lower().eq("total")
].copy()

# Convert to GWh and build yearly totals by grouped fuel
ca["GENERATION_GWH"] = pd.to_numeric(ca[gen_col], errors="coerce") / 1_000
ca_yearly = (
    ca.groupby(["YEAR", "ENERGY SOURCE Grouped"], as_index=False)["GENERATION_GWH"]
      .sum()
      .sort_values(["YEAR","ENERGY SOURCE Grouped"])
)

# Governors (start years only)
gov_ca = prep_governors_ca(gov)

# --------------------------- Figure 3.1 — Absolute GWh by source ---------------------------
fig_31 = px.line(
    ca_yearly, x="YEAR", y="GENERATION_GWH", color="ENERGY SOURCE Grouped",
    markers=True,
    labels={"YEAR":"Year","GENERATION_GWH":"Generation (GWh)","ENERGY SOURCE Grouped":"Energy Source"},
    title="Figure 3.1 — California Electricity Generation by Source (GWh), 2001–2019"
).update_layout(template="plotly_white", xaxis=dict(dtick=1))
fig_31 = add_governor_overlays(fig_31, gov_ca)  # yref=paper so labels don't clip
fig_31.show()

# --------------------------- Figure 3.2 — YoY % change per source ---------------------------
ca_yoy = ca_yearly.sort_values(["ENERGY SOURCE Grouped","YEAR"]).copy()
ca_yoy["YoY %"] = ca_yoy.groupby("ENERGY SOURCE Grouped")["GENERATION_GWH"].pct_change() * 100

fig_32 = px.line(
    ca_yoy.dropna(subset=["YoY %"]),
    x="YEAR", y="YoY %", color="ENERGY SOURCE Grouped", markers=True,
    labels={"YEAR":"Year","YoY %":"Year-over-Year Change (%)","ENERGY SOURCE Grouped":"Energy Source"},
    title="Figure 3.2 — California YoY % Change in Generation by Source (2002–2019)"
).update_layout(template="plotly_white", xaxis=dict(dtick=1))
fig_32 = add_governor_overlays(fig_32, gov_ca)
fig_32.show()

# --------------------------- Figure 3.3 — % of total (mix) ---------------------------
share = ca_yearly.copy()
share["CA_TOTAL_GWH"] = share.groupby("YEAR")["GENERATION_GWH"].transform("sum")
share["Share (%)"] = 100 * share["GENERATION_GWH"] / share["CA_TOTAL_GWH"]

fig_33 = px.area(
    share.sort_values(["YEAR","ENERGY SOURCE Grouped"]),
    x="YEAR", y="Share (%)", color="ENERGY SOURCE Grouped",
    labels={"YEAR":"Year","Share (%)":"Share of California Generation (%)","ENERGY SOURCE Grouped":"Energy Source"},
    title="Figure 3.3 — California Generation Mix (% of Total), 2001–2019"
).update_layout(template="plotly_white", xaxis=dict(dtick=1), yaxis=dict(range=[0,100]))
fig_33 = add_governor_overlays(fig_33, gov_ca)
fig_33.show()

# --------------------------- Figure 3.4 — Low-carbon only ---------------------------
LOW_CARBON = {"Geothermal", "Hydroelectric Conventional", "Solar Thermal and Photovoltaic", "Wind"}
lc = ca_yearly[ca_yearly["ENERGY SOURCE Grouped"].isin(LOW_CARBON)].copy()

fig_34 = px.line(
    lc.sort_values(["YEAR","ENERGY SOURCE Grouped"]),
    x="YEAR", y="GENERATION_GWH", color="ENERGY SOURCE Grouped", markers=True,
    labels={"YEAR":"Year","GENERATION_GWH":"Generation (GWh)","ENERGY SOURCE Grouped":"Energy Source"},
    title="Figure 3.4 — California Low-Carbon Generation (Geothermal, Hydro, Solar, Wind), 2001–2019"
).update_layout(template="plotly_white", xaxis=dict(dtick=1))
fig_34 = add_governor_overlays(fig_34, gov_ca)
fig_34.show()


Figures 3.5 - 3.8 show the same analysis applied to West Virginia, also known as "The Coal State." Indeed, coal generation fulfilled over 90% of the state's electricity demand, though it has been decreasing marginally throughout the 21st century. Hydropower, wind, and natural gas have made gains, but have not come close to replacing coal's dominance in the state. In fact, these sources have not filled in the hole that has been left by decreasing coal generation. Table 1 shows that overall electricity generation has decreased by over 20% between 2001 to 2019. Finally, changes in governorship seem to have had little impact, with no noticeable changes in trajectory under changing leadership.

In [ ]:
import pandas as pd
import plotly.express as px

# ---------- Helpers (safe to re-run) ----------
def detect_gen_col(df):
    for c in df.columns:
        if "GENERATION" in c.upper():
            return c
    raise KeyError("Generation column not found (expected 'GENERATION (MWh)' or similar).")

def ensure_grouped_column(df):
    """Add ENERGY SOURCE Grouped, collapsing the five 'Other Sources'."""
    if "ENERGY SOURCE Grouped" in df.columns:
        return df
    other_sources = {"wood and wood derived fuels","other biomass","other","other gases","pumped storage"}
    out = df.copy()
    out["ENERGY SOURCE Grouped"] = out["ENERGY SOURCE"].apply(
        lambda x: "Other Sources" if str(x).strip().lower() in other_sources else x
    )
    return out

def prep_governors(gov_df, state_name):
    """Keep only the first (start) year of each governor's term for a state, 2001–2019."""
    party_colors = {"Democratic": "blue", "Republican": "red", "Other": "gray"}
    g = gov_df[gov_df["State"].str.lower().eq(state_name.lower())].copy()
    g = g[g["Year"].between(2001, 2019)]
    g = g.sort_values(["Governor", "Year"]).groupby("Governor", as_index=False).first()
    g["Color"] = g["Party"].map(party_colors).fillna("gray")
    return g

def add_governor_overlays(fig, gov_state, yref="paper", y=0.97):
    """Add dashed vertical lines and labeled annotations that won't clip off."""
    for _, r in gov_state.iterrows():
        year  = int(r["Year"])
        color = r["Color"]
        name  = str(r["Governor"]).strip()
        party = str(r.get("Party","Other")).strip()
        fig.add_vline(x=year, line_width=2, line_dash="dash", line_color=color)
        fig.add_annotation(
            x=year, y=y, yref=yref,
            text=f"{name} ({party[0]})",
            showarrow=False,
            font=dict(size=10, color="white"),
            bgcolor=color, bordercolor="black", borderwidth=0.5, borderpad=2,
            textangle=90, yanchor="top"
        )
    return fig

# ---------- Data prep (West Virginia) ----------
gen_col = detect_gen_col(gen)
gen = ensure_grouped_column(gen)

wv = gen[
    gen["STATE"].str.upper().eq("WV")
    & gen["YEAR"].between(2001, 2019)
    & ~gen["ENERGY SOURCE"].astype(str).str.strip().str.lower().eq("total")
].copy()

wv["GENERATION_GWH"] = pd.to_numeric(wv[gen_col], errors="coerce") / 1_000
wv_yearly = (
    wv.groupby(["YEAR", "ENERGY SOURCE Grouped"], as_index=False)["GENERATION_GWH"]
      .sum()
      .sort_values(["YEAR","ENERGY SOURCE Grouped"])
)

gov_wv = prep_governors(gov, "West Virginia")

# ---------- Figure 3.5 — WV Absolute GWh by source ----------
fig_35 = px.line(
    wv_yearly, x="YEAR", y="GENERATION_GWH", color="ENERGY SOURCE Grouped",
    markers=True,
    labels={"YEAR":"Year","GENERATION_GWH":"Generation (GWh)","ENERGY SOURCE Grouped":"Energy Source"},
    title="Figure 3.5 — West Virginia Electricity Generation by Source (GWh), 2001–2019"
).update_layout(template="plotly_white", xaxis=dict(dtick=1))
fig_35 = add_governor_overlays(fig_35, gov_wv)
fig_35.show()

# ---------- Figure 3.6 — WV YoY % change per source ----------
wv_yoy = wv_yearly.sort_values(["ENERGY SOURCE Grouped","YEAR"]).copy()
wv_yoy["YoY %"] = wv_yoy.groupby("ENERGY SOURCE Grouped")["GENERATION_GWH"].pct_change() * 100

fig_36 = px.line(
    wv_yoy.dropna(subset=["YoY %"]),
    x="YEAR", y="YoY %", color="ENERGY SOURCE Grouped", markers=True,
    labels={"YEAR":"Year","YoY %":"Year-over-Year Change (%)","ENERGY SOURCE Grouped":"Energy Source"},
    title="Figure 3.6 — West Virginia YoY % Change in Generation by Source (2002–2019)"
).update_layout(template="plotly_white", xaxis=dict(dtick=1))
fig_36 = add_governor_overlays(fig_36, gov_wv)
fig_36.show()

# ---------- Figure 3.7 — WV % of total (mix) ----------
share_wv = wv_yearly.copy()
share_wv["WV_TOTAL_GWH"] = share_wv.groupby("YEAR")["GENERATION_GWH"].transform("sum")
share_wv["Share (%)"] = 100 * share_wv["GENERATION_GWH"] / share_wv["WV_TOTAL_GWH"]

fig_37 = px.area(
    share_wv.sort_values(["YEAR","ENERGY SOURCE Grouped"]),
    x="YEAR", y="Share (%)", color="ENERGY SOURCE Grouped",
    labels={"YEAR":"Year","Share (%)":"Share of West Virginia Generation (%)","ENERGY SOURCE Grouped":"Energy Source"},
    title="Figure 3.7 — West Virginia Generation Mix (% of Total), 2001–2019"
).update_layout(template="plotly_white", xaxis=dict(dtick=1), yaxis=dict(range=[0,100]))
fig_37 = add_governor_overlays(fig_37, gov_wv)
fig_37.show()

# ---------- Figure 3.8 — WV Low-carbon only ----------
LOW_CARBON = {"Geothermal", "Hydroelectric Conventional", "Solar Thermal and Photovoltaic", "Wind"}
wv_lc = wv_yearly[wv_yearly["ENERGY SOURCE Grouped"].isin(LOW_CARBON)].copy()

fig_38 = px.line(
    wv_lc.sort_values(["YEAR","ENERGY SOURCE Grouped"]),
    x="YEAR", y="GENERATION_GWH", color="ENERGY SOURCE Grouped", markers=True,
    labels={"YEAR":"Year","GENERATION_GWH":"Generation (GWh)","ENERGY SOURCE Grouped":"Energy Source"},
    title="Figure 3.8 — West Virginia Low-Carbon Generation (Geothermal, Hydro, Solar, Wind), 2001–2019"
).update_layout(template="plotly_white", xaxis=dict(dtick=1))
fig_38 = add_governor_overlays(fig_38, gov_wv)
fig_38.show()

#generating table to assess overall electricity production

import pandas as pd
from IPython.display import display, Markdown

# --- Detect generation column ---
gen_col = next((c for c in gen.columns if "GENERATION" in c.upper()), None)
if gen_col is None:
    raise KeyError("Generation column not found (expected 'GENERATION (MWh)' or similar).")

# --- West Virginia subset (exclude 'Total' energy-source rows) ---
wv = gen[
    gen["STATE"].str.upper().eq("WV")
    & gen["YEAR"].between(2001, 2019)
    & ~gen["ENERGY SOURCE"].astype(str).str.strip().str.lower().eq("total")
].copy()

# Convert to GWh
wv["GENERATION_GWH"] = pd.to_numeric(wv[gen_col], errors="coerce") / 1_000

# Sum totals for 2001 and 2019
totals = (
    wv[wv["YEAR"].isin([2001, 2019])]
      .groupby("YEAR", as_index=True)["GENERATION_GWH"]
      .sum()
)

# Build a single-row table (handles missing years gracefully)
row_2001 = totals.get(2001, float("nan"))
row_2019 = totals.get(2019, float("nan"))
tbl = pd.DataFrame({
    "2001 (GWh)": [row_2001],
    "2019 (GWh)": [row_2019],
})

# Format with commas, no decimals
for col in ["2001 (GWh)", "2019 (GWh)"]:
    if pd.notnull(tbl.loc[0, col]):
        tbl.loc[0, col] = f"{int(round(tbl.loc[0, col])):,}"
    else:
        tbl.loc[0, col] = "—"

display(Markdown("### Table 1 — West Virginia: Total Electricity Generation, 2001 vs 2019"))
display(tbl)




/tmp/ipython-input-74226509.py:156: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '81,837' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

/tmp/ipython-input-74226509.py:156: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '63,926' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



### Table 1 — West Virginia: Total Electricity Generation, 2001 vs 2019

,2001 (GWh),2019 (GWh)
0,"81,837","63,926"


Figures 3.9 - 3.12 show changes in the Texas electric system from 2001 to 2019. Texas, home to the Permian Basin which boasts significant amount of the world's natural gas preserves, has a reputation as one of the strongest fossil fuel production regions in the world. The 2000s were defined by a dominance of natural gas and coal. In the latter half of the decade, wind began increasing its output. In the 2010s, the convergence of coal and natural gas began, with coal losing market share while natural gas increased its generation to maintain roughly 50% generation in the state. While wind ended the 2010s with a nearly 18% market share, other low-carbon technologies like solar, geothermal, and hydropower remained mostly irrelevant for electricity generation in Texas.

An interesting observable phenomenon upon the start of Abbott's governorship in 2015 was the change in trajectory for several generation sources, including natural gas, which began a new downward trajectory, and wind, which saw its best years in the years following Abbott's start.

In [ ]:
#Adding Greg Abbott (R, 2015)

# --- Add Greg Abbott manually if missing ---
# (Safe: will not duplicate if record already exists)
if not ((gov["Governor"] == "Greg Abbott") & (gov["State"] == "Texas")).any():
    gov = pd.concat([
        gov,
        pd.DataFrame({
            "Governor": ["Greg Abbott"],
            "State": ["Texas"],
            "Time In Office": ["2015–present"],  # optional detail
            "Party": ["Republican"],
            "Year": [2015]
        })
    ], ignore_index=True)

print("✅ Greg Abbott added to governor list (if not already present).")


import pandas as pd
import plotly.express as px

# ---------- Helper functions (reuse from earlier) ----------
def detect_gen_col(df):
    for c in df.columns:
        if "GENERATION" in c.upper():
            return c
    raise KeyError("Generation column not found (expected 'GENERATION (MWh)' or similar).")

def ensure_grouped_column(df):
    """Add ENERGY SOURCE Grouped, collapsing five minor categories into 'Other Sources'."""
    if "ENERGY SOURCE Grouped" in df.columns:
        return df
    other_sources = {"wood and wood derived fuels","other biomass","other","other gases","pumped storage"}
    out = df.copy()
    out["ENERGY SOURCE Grouped"] = out["ENERGY SOURCE"].apply(
        lambda x: "Other Sources" if str(x).strip().lower() in other_sources else x
    )
    return out

def prep_governors(gov_df, state_name):
    """Keep only the first (start) year of each governor's term for a state, 2001–2019."""
    party_colors = {"Democratic": "blue", "Republican": "red", "Other": "gray"}
    g = gov_df[gov_df["State"].str.lower().eq(state_name.lower())].copy()
    g = g[g["Year"].between(2001, 2019)]
    g = g.sort_values(["Governor", "Year"]).groupby("Governor", as_index=False).first()
    g["Color"] = g["Party"].map(party_colors).fillna("gray")
    return g

def add_governor_overlays(fig, gov_state, yref="paper", y=0.97):
    """Add dashed vertical lines and name labels that won't clip."""
    for _, r in gov_state.iterrows():
        year  = int(r["Year"])
        color = r["Color"]
        name  = str(r["Governor"]).strip()
        party = str(r.get("Party","Other")).strip()
        fig.add_vline(x=year, line_width=2, line_dash="dash", line_color=color)
        fig.add_annotation(
            x=year, y=y, yref=yref,
            text=f"{name} ({party[0]})",
            showarrow=False,
            font=dict(size=10, color="white"),
            bgcolor=color, bordercolor="black", borderwidth=0.5, borderpad=2,
            textangle=90, yanchor="top"
        )
    return fig

# ---------- Texas data prep ----------
gen_col = detect_gen_col(gen)
gen = ensure_grouped_column(gen)

tx = gen[
    gen["STATE"].str.upper().eq("TX")
    & gen["YEAR"].between(2001, 2019)
    & ~gen["ENERGY SOURCE"].astype(str).str.strip().str.lower().eq("total")
].copy()

tx["GENERATION_GWH"] = pd.to_numeric(tx[gen_col], errors="coerce") / 1_000
tx_yearly = (
    tx.groupby(["YEAR", "ENERGY SOURCE Grouped"], as_index=False)["GENERATION_GWH"]
      .sum()
      .sort_values(["YEAR","ENERGY SOURCE Grouped"])
)

gov_tx = prep_governors(gov, "Texas")

# ---------- Figure 3.9 — Absolute GWh by source ----------
fig_38 = px.line(
    tx_yearly, x="YEAR", y="GENERATION_GWH", color="ENERGY SOURCE Grouped",
    markers=True,
    labels={"YEAR":"Year","GENERATION_GWH":"Generation (GWh)","ENERGY SOURCE Grouped":"Energy Source"},
    title="Figure 3.9 — Texas Electricity Generation by Source (GWh), 2001–2019"
).update_layout(template="plotly_white", xaxis=dict(dtick=1))
fig_38 = add_governor_overlays(fig_38, gov_tx)
fig_38.show()

# ---------- Figure 3.10 — YoY % change per source ----------
tx_yoy = tx_yearly.sort_values(["ENERGY SOURCE Grouped","YEAR"]).copy()
tx_yoy["YoY %"] = tx_yoy.groupby("ENERGY SOURCE Grouped")["GENERATION_GWH"].pct_change() * 100

fig_39 = px.line(
    tx_yoy.dropna(subset=["YoY %"]),
    x="YEAR", y="YoY %", color="ENERGY SOURCE Grouped", markers=True,
    labels={
        "YEAR": "Year",
        "YoY %": "Year-over-Year Change (%)",
        "ENERGY SOURCE Grouped": "Energy Source"
    },
    title="Figure 3.10 — Texas YoY % Change in Generation by Source (2002–2019)"
)

# Set style, ticks, and y-axis limits ±100%
fig_39.update_layout(
    template="plotly_white",
    xaxis=dict(dtick=1),
    yaxis=dict(range=[-200, 200])   # 👈 constrain to ±100%
)

fig_39 = add_governor_overlays(fig_39, gov_tx)
fig_39.show()


# ---------- Figure 3.11 — % of total (mix) ----------
share_tx = tx_yearly.copy()
share_tx["TX_TOTAL_GWH"] = share_tx.groupby("YEAR")["GENERATION_GWH"].transform("sum")
share_tx["Share (%)"] = 100 * share_tx["GENERATION_GWH"] / share_tx["TX_TOTAL_GWH"]

fig_310 = px.area(
    share_tx.sort_values(["YEAR","ENERGY SOURCE Grouped"]),
    x="YEAR", y="Share (%)", color="ENERGY SOURCE Grouped",
    labels={"YEAR":"Year","Share (%)":"Share of Texas Generation (%)","ENERGY SOURCE Grouped":"Energy Source"},
    title="Figure 3.11 — Texas Generation Mix (% of Total), 2001–2019"
).update_layout(template="plotly_white", xaxis=dict(dtick=1), yaxis=dict(range=[0,100]))
fig_310 = add_governor_overlays(fig_310, gov_tx)
fig_310.show()

# ---------- Figure 3.12 — Low-carbon only ----------
LOW_CARBON = {"Geothermal", "Hydroelectric Conventional", "Solar Thermal and Photovoltaic", "Wind"}
tx_lc = tx_yearly[tx_yearly["ENERGY SOURCE Grouped"].isin(LOW_CARBON)].copy()

fig_311 = px.line(
    tx_lc.sort_values(["YEAR","ENERGY SOURCE Grouped"]),
    x="YEAR", y="GENERATION_GWH", color="ENERGY SOURCE Grouped", markers=True,
    labels={"YEAR":"Year","GENERATION_GWH":"Generation (GWh)","ENERGY SOURCE Grouped":"Energy Source"},
    title="Figure 3.12 — Texas Low-Carbon Generation (Geothermal, Hydro, Solar, Wind), 2001–2019"
).update_layout(template="plotly_white", xaxis=dict(dtick=1))
fig_311 = add_governor_overlays(fig_311, gov_tx)
fig_311.show()


✅ Greg Abbott added to governor list (if not already present).


**4. Conclusion**

The data from the Energy Information Agency has the potential for endless exploration and analysis. This project took a high level approach to understand some of the nuances of electricity production in the United States broadly, followed by the analysis of California, West Virginia, and Texas, three states that have their own unique reputations in the energy industry.

This analysis highlights the challenge of decarbonizing electricity generation across the country. Generation is certainly not monolithic, with great variance existing across the country based on the natural resources available to each state. All of them, however, must prioritize the decarbonization of their electiricty grids if the U.S. economy is to become a leader in systemic decarbonization. There will be no one-size-fits-all solutions, and just as regions have played to their strenghts in their fossil-fuel-dependent histories, they must also play to their strengths when it comes to low-carbon generation, considering the strengths and drawbacks of solar, wind, geothermal, batteries, pumped storage, and other technologies based on their natural endowment, as well as the concentration of electricity demand throughout the state.

This project could be improved by creating an interactive dashboard where the figures throughout could be actively filtered and toggeled in order to switch between states' data. It could also benefit from the inclusion of distributed sources, which is particularly relevant for small-scale technologies like solar PV.

**5. Sources**

**Data Sources**
US Generation Data: https://www.kaggle.com/datasets/kevinmorgado/us-energy-generation-2001-2022

US Governors Data: https://www.openicpsr.org/openicpsr/project/102000/version/V3/view

**Sources**

Hannah Ritchie, Our World in Data (2025), "Which countries have contributed the most to historical CO₂ emissions?," https://ourworldindata.org/data-insights/which-countries-have-contributed-the-most-to-historical-co-emissions.

Environmental Portection Agency, "Electric Power Sector Emissions," https://www.epa.gov/ghgemissions/electric-power-sector-emissions.

International Energy Agency, "United States -- Electricity Generation," https://www.iea.org/countries/united-states/electricity.